In [2]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import json
from networkx.readwrite import json_graph


In [3]:
import findspark
findspark.find()
findspark.init(edit_profile=True)
import pyspark
sc = pyspark.SparkContext()

In [3]:
with open("../../graph/nc_full.json", "r") as graph_data:
    nc_full_data = json.load(graph_data)
    nc_full_digraph = json_graph.node_link_graph(nc_full_data)
    

In [6]:
with open("../../graph/nc_mini.json", "r") as graph_data:
    nc_mini_data = json.load(graph_data)
    nc_mini_digraph = json_graph.node_link_graph(nc_mini_data)
    

In [4]:
nc_full_empty_nodes = [i for i in nc_full_digraph.nodes() if not nc_full_digraph.succ[i]]

In [19]:
nc_full_N_width_nodes = [i for i in nc_full_digraph.nodes() if len(nc_full_digraph.succ[i]) >= 50]

In [9]:
nc_mini_N_width_nodes = [i for i in nc_mini_digraph.nodes() if len(nc_mini_digraph.succ[i]) >= 10]

In [7]:
# nc_full_N_depth_nodes = [i for i in nc_full_digraph.nodes() if len(nx.dfs_successors(nc_full_digraph, i)) >= 5]

In [ ]:
nc_full_N_depth_width_nodes = [i for i in nc_full_digraph.nodes() 
                          if len(nx.dfs_successors(nc_full_digraph, i)) >= 5 & len(nc_full_digraph.succ[i]) >= 200]

In [22]:
# len(nc_full_N_depth_nodes)

In [21]:
len(nc_full_N_width_nodes)

212

In [10]:
len(nc_mini_N_width_nodes)

32

In [14]:
start = time.time()
mini_influence_list = {}
for i in nc_mini_N_width_nodes:
    mini_influence_list[i] = influence_function(100, [i], 16)
    print i, ':', mini_influence_list[i]
sorted_max_potential_nodes = sorted(mini_influence_list.items(), key=lambda x: x[1], reverse=True )

print 'max potential node set', sorted_max_potential_nodes[:3]

PpkKVodWC0sdn74TbHQLzA : 12
emoQY8neOSJm-xwqh4xVfQ : 17
glzRad3_l42hALvjB9p97w : 12
M9H6p4hm2utu1G0ipV80Ug : 10
W6LTsXcY1R76M9EdeN0G6A : 18
XZuMpc6E9RqZX4HXtmlr-g : 9
DK2gGNdN3ihOu_uwt2AgLg : 13
A6bPFcUjuuayRBoyybedDQ : 20
M-TwsqjrGVH9-qyw2KcvdQ : 25
Kybm9SJyOfEgDAXFUPRdCQ : 12
NzWLMPvbEval0OVg_YDn4g : 27
-_1ctLaz3jhPYc12hKXsEQ : 22
1K50PvXa6l5PBXZXhzkB-g : 10
whKpnugMnWTz3UjlrN71yg : 14
do-fO-EruYVP_FQPMNWmZg : 21
Y--yRCncNlQCNULP8I5h6Q : 12
7ewE_dDxjXNppOj0EhhMIQ : 15
3GteArKIVRlJZjjRqqwgaQ : 20
LP0bl3GKToABt3lnpSvA2w : 22
tm_HvdZmUMGht1-xjGQF7Q : 11
eFxX1tSNLmVkAP9kNv2--Q : 19
aZjcmboRYOYyfmD6AdLesQ : 12
s9I4KMcrHKki44g_eTrYqQ : 23
ED6n40WmZJm0AvsKkG7iaw : 19
Q9xJQu-9oCFZozVBjZDETw : 15
Ul93T_U5jDh1FIIpiiTxuw : 13
VhI6xyylcAxi0wOy2HOX3w : 24
ts7EG6Zv2zdMDg29nyqGfA : 28
cZjX4HsWB8mTLHI4uhbB-g : 10
tFLi8-M6MOOrY5CsvWHRMA : 14
lpZtfD-vCqUER1yGKEvUPw : 23
gD2FceWxl711U2IJokXBAQ : 8
max potential node set [(u'ts7EG6Zv2zdMDg29nyqGfA', 28), (u'NzWLMPvbEval0OVg_YDn4g', 27), (u'M-TwsqjrGVH9-

In [15]:
influence_function(100, [u'ts7EG6Zv2zdMDg29nyqGfA', u'NzWLMPvbEval0OVg_YDn4g', u'M-TwsqjrGVH9-qyw2KcvdQ'], 16)

42

In [ ]:
len(nc_full_N_depth_width_nodes)

In [13]:
start = time.time()
influence_list = {}
for i in nc_full_N_width_nodes:
    influence_list[i] = influence_function(100, [i], 16)
    print i, ':', influence_list[i]
sorted_max_potential_nodes = sorted(influence_list.items(), key=lambda x: x[1], reverse=True )

print 'max potential node set', sorted_max_potential_nodes[:3]

NameError: name 'nc_full_N_width_nodes' is not defined

In [29]:
influence_function(100, ['4G68oLRY3aHE5XUt_MUUcA', 'CvMVd31cnTfzMUsHDXm4zQ', 'NzWLMPvbEval0OVg_YDn4g'], 16)

8964

In [12]:
def cascade(init_nodes, nodes_set_broadcast):#, dist_d):
    nodes_set = nodes_set_broadcast.value
    action = {}
    n = len(init_nodes)
    #np.random.seed(random_d)
    #init_nodes = np.random.choice(NC_digraph.nodes(), 1)[0]
    for i in init_nodes:
        action[i] = 1
    #st = set()
    #st.add(init_nodes)
    init_list = zip([0]*len(init_nodes),init_nodes[::])
    inter = 0
    while len(init_list) != 0:
        curr_node = init_list.pop(0)
        #print curr_node
        for i in nodes_set[curr_node[1]]:
            if i not in action:
                b = nodes_set.node[i]['review_count']
                a =  nodes_set[curr_node[1]][i]['weight']
                #np.random.seed(12)
                b_dist = np.sqrt(np.random.beta(a = a, b = b))
                #np.random.seed(12)
                u_dist = np.random.uniform(0,1)
                if b_dist > u_dist:
                    action[i] = 1
                    #st.add(i)
                    inter = curr_node[0] + 1
                    init_list.append((inter, i))
                    n = n + 1
    return n

def influence_function(N, init_nodes, partition_num):
    nodes_set_broadcast = sc.broadcast(nc_full_digraph)
    activated_num_rdd = sc.parallelize([init_nodes]*N, partition_num)
    activated_num = activated_num_rdd.map(lambda x: cascade(x, nodes_set_broadcast)).reduce(lambda x, y: x+y)
    return activated_num/N
####################################################################################


######################################################################################
#
#Simulated Annealing Algorithm Implementation
#
######################################################################################

def next_step(X, nodes_set):
    tmp = X[0]
    while tmp in X:
        tmp = np.random.choice(nodes_set)
    X[np.random.choice(range(len(X)))] = tmp
    return X


def simulated_annealing_rst(function, N, partition_num, initial_X, initial_temp, cool, reanneal, iterr, nodes_set):
    
    accepted = 0
    #X = initial_X.copy()
    X = initial_X[::]
    T = initial_temp
    
    history = list()
    # Evaluate E
    prev_E = function(N, X, partition_num)
    history.append(prev_E)
    
    for i in xrange(iterr):
        # Propose new path.
        X_star = next_step(X, nodes_set)
        # Evaluate E
        new_E = function(N, X_star, partition_num)
        delta_E = new_E - prev_E
        
        # Flip a coin
        U = np.random.uniform()
        
        if U < np.exp(delta_E / T):
            accepted += 1
            history.append(new_E)
            print 'value:', new_E
            # Copy X_star to X
            print "this is", i, "iteration"
            #X = X_star.copy()
            X = X_star[::]
            print 'set:', X
            prev_E = new_E
        
        # Check to cool down
        if accepted % reanneal == 0:
            T *= cool
    return X, history

#Finding starting temperature

In [ ]:
def influence_function_for_finding_T(N, init_nodes, partition_num):
    nodes_set_broadcast = sc.broadcast(nc_full_digraph)
    activated_num_rdd = sc.parallelize([init_nodes]*N, partition_num)
    activated_num = activated_num_rdd.map(lambda x: cascade(x, nodes_set_broadcast)).reduce(lambda x, y: x+y)
    return activated_num/N

In [ ]:
start = time.time()
influence_list = {}
for i in top_nodes:
    influence_list[i] = cascade([i], nc_full_digraph)
    print i, ':', influence_list[i]
sorted_max_potential_nodes = sorted(influence_list.items(), key=lambda x: x[1], reverse=True )

top_nodeset = sorted_max_potential_nodes[:3]
least_nodeset = sorted_max_potential_nodes[:-3]

max_influence = influence_function_for_finding_T(1000, top_nodeset, 16)
min_influence = influence_function_for_finding_T(1000, least_nodeset, 16)

difference_range= max_influence - min_influence